*Copyright 2024. Fermi Research Alliance, LLC. This material is based upon work supported by the U.S. Department of Energy, Office of Science, National Quantum Information Science Research Centers, Superconducting Quantum Materials and Systems (SQMS) Center under the contract No. DE-AC02-07CH11359. The U.S. Government has rights to use, reproduce, and distribute this software. NEITHER THE GOVERNMENT NOR FERMI RESEARCH ALLIANCE, LLC MAKES ANY WARRANTY, EXPRESS OR IMPLIED, OR ASSUMES ANY LIABILITY FOR THE USE OF THIS SOFTWARE. If software is modified to produce derivative works, such modified software should be clearly marked, so as not to confuse it with the version available from Fermilab.*

*Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0*

*Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Quantum state & process tomography

- Author: Nicholas Bornman
- Date: 21 August 2024

This notebook contains functions and logic needed to perform single and two qubit quantum state tomography using just the probability estimators we'd get from actual lab results as the input, as well as single and two qubit standard quantum process tomography, also using only the experimental results as input.

The code also optionally prints useful descriptions of the steps in important subroutines, such as the kinds of separable states that need to be prepared for the single and two qubit cases, and/or the basis in which the local measurements should be performed.

Useful references:

1. https://research.physics.illinois.edu/QI/Photonics/tomography-files/tomo_chapter_2004.pdf
2. https://arxiv.org/pdf/quant-ph/0702131
3. https://arxiv.org/pdf/1509.02921

In [ ]:
should_print_description = True

## Preamble

In [ ]:
import os
import numpy as np
import functools
import pickle

from scipy.linalg import expm, ishermitian, solve, eig

import matplotlib

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm, colormaps


Matplotlib plot settings

In [ ]:
titlefont = {'color':  'black', 'weight': 'normal','size': 10}
axisfont = {'color':  'black', 'weight': 'normal','size': 8}
ticksfont = {'color':  'black', 'weight': 'normal','size': 8}

Some useful defintions

In [ ]:
x = np.array([[0,1],[1,0]])
y = np.array([[0,-1j],[1j,0]])
z = np.array([[1,0],[0,-1]])
i = np.array([[1,0],[0,1]])
h = (1/np.sqrt(2)) * np.array([[1,1],[1,-1]])

In [ ]:
def matrix_rotate(theta, op):

    return expm((-1j/2) * theta * op)

In [ ]:
xm90 = matrix_rotate(-np.pi/2, x)
x90 = matrix_rotate(np.pi/2, x)
ym90 = matrix_rotate(-np.pi/2, y)
y90 = matrix_rotate(np.pi/2, y)

In [ ]:
def ketbra(ket, bra):

    return np.outer(ket, np.conj(bra))

def ketbradiag(ket):

    return np.outer(ket, np.conj(ket))

def kron(op1, op2):

    return np.kron(op1, op2)

In [ ]:
def is_pos_semidef(x):
    return np.all(np.linalg.eigvals(x) >= 0)

## Quantum State Tomography

The following sections have detailed explanations of the purpose of the various functions and other Python objects, but the math and logic below follows Ref 1 closely. The reader is encouraged to consult this reference.

Run the "Single qubit functions" and "Two qubit functions" sections to load the required functions needed in the examples sections.

What you need to know for the "single qubit" section:
1. For any prepared state, in the lab, take measurements of the qubit in the $X$, $Y$ and $Z$ bases. This gives you the probabilities of measuring the six cardinal Bloch sphere states of the qubit, before readout correction
2. Optionally, use the results from a calibrated two state discriminator to correct any readout error in the results
3. Format these results into a results dictionary similar to that outputted by the 'compute_probabilities1' function in the "Single qubit example" section
4. Run 'compute_stokes_parameters1' on this object, which gives the corresponding Stokes' parameters dictionary
5. Run "reconstruct_state1" on this dictionary, which gives you the inferred density matrix for the state

What you need to know for the "two qubit" section:
1. For any prepared two qubit state, in the lab, take measurements of the qubits in their $X$, $Y$ and $Z$ bases, independently (so you perform measurements in 3x3 = 9 different bases). This gives you the probabilities of measuring the qubits in their individual six Bloch sphere states, before readout correction
2. Optionally, use the results from a calibrated two state discriminator, one for each of the two qubits, to correct any readout error in the results
3. Format these results into a results dictionary similar to that outputted by the 'compute_probabilities2' function in the "Two qubit example" section
4. Run 'compute_stokes_parameters2' on this object, which gives the corresponding Stokes' parameters dictionary
5. Run "reconstruct_state2" on this dictionary, which gives you the inferred density matrix for the two qubit state

### Single qubit functions

Maps an integer to a string representing one of the six cardinal points of the Bloch sphere. Pairs 1-2, 3-4, and 5-6, are complementary states and each form a basis for a single qubit Hilbert space.

In [ ]:
integer_to_state_string_map1 = {
    1: "|0>",
    2: "|1>",
    3: "1/sqrt(2)(|0>+|1>)",
    4: "1/sqrt(2)(|0>-|1>)",
    5: "1/sqrt(2)(|0>+i|1>)",
    6: "1/sqrt(2)(|0>-i|1>)"
}

Maps an integer to a numpy array representation of the qubit state corresponding to the string representation from the above dictionary.

In [ ]:
integer_to_state_ket_map1 = {
    1: np.array([1, 0]),
    2: np.array([0, 1]),
    3: 1/np.sqrt(2) * np.array([1, 1]),
    4: 1/np.sqrt(2) * np.array([1, -1]),
    5: 1/np.sqrt(2) * np.array([1, 1j]),
    6: 1/np.sqrt(2) * np.array([1, -1j])
}

A map that indicates the ideal single qubit gate that needs to be applied to a qubit initialised in the ground state, in order to create the state with the string representation from the 'integer_to_state_string_map1' map above, or the numpy array representation from the 'integer_to_state_ket_map1' map above.

In [ ]:
make_state_from_ground_map1 = {
    1: "I",
    2: "X180/Y180",
    3: "Y90",
    4: "Y-90",
    5: "X-90",
    6: "X90"
}

Maps the stokes index ($i$ in $S_{i}$, where $S_{i}$ is a Stokes parameter) to the Pauli operator basis decomposisiton of any density matrix

In [ ]:
stokes_index_to_pauli1 = {0: i, 1: x, 2: y, 3: z}

A dictionary which maps the Stokes parameter index to a list of two lists. The first is a list of integers corresponding to Bloch sphere states (as in the 'integer_to_state_string_map1' map above), projective measurement values of which need to be ADDED together, while the second is a list of integers corresponding to Bloch sphere states, projective measurement values of which need to be SUBTRACTED, in order to reconstruct the Stokes parameter.

For example, given an arbitrary unknown density matrix, to deduce the Stokes parameter $S_{1}$, we would need to measure the probability of measuring the Bloch sphere state 3, i.e. $\frac{1}{\sqrt{2}} \left(\left |0 \right > + \left |1 \right > \right)$ (giving $P_3$), minus the probability of measuring the Bloch sphere state 4, i.e. $\frac{1}{\sqrt{2}} \left(\left |0 \right > - \left |1 \right > \right)$ (giving $P_4$). Then $S_{1} = P_3 - P_4$. Similarly, to deduce the Stokes parameter $S_{0}$, we would need to measure the probability of measuring the Bloch sphere state 1, i.e. $\left |0 \right >$ (giving $P_1$), plus the probability of measuring the Bloch sphere state 2, i.e. $\left |1 \right >$ (giving $P_2$). Then $S_{0} = P_1 + P_2$ (which should always equal 1 for a pure state).

In [ ]:
stokes_index_to_projector_combinations_map1 = {0: [[1,2], []], 1: [[3], [4]], 2: [[5], [6]], 3: [[1], [2]]}

Given a single qubit density matrix, the following function decomposes the state into the Pauli basis and outputs a dictionary which maps the Stokes parameter index to the corresponding value.

This function is used in simulations and can be used in sanity checks.

In [ ]:
def compute_probabilities1(rho, should_print_description = False):

    assoc = {}

    if should_print_description:
        print(f"For input state rho = {rho}:\n")
    
    for j in range(1, len(integer_to_state_string_map1) + 1):
        answer = np.round(np.trace(rho @ ketbradiag(integer_to_state_ket_map1[j])), decimals=7)

        if not np.isclose(answer.imag, 0.0):
            raise ValueError(f"The probability value {anwser} should have an imaginary part of 0.")
        else:
            answer = answer.real

        if should_print_description:
            print(f"{j}: Probability of measuring state {integer_to_state_string_map1[j]} (which is made using gate {make_state_from_ground_map1[j]} from |0>): {answer}\n")
        
        assoc[j] = answer
    
    return assoc

Given an input dictionary with keys corresponding to the integers of the six cardinal Bloch sphere states (as in the dictionary 'integer_to_state_string_map1') and the values corresponding with the probability of measuring the corresponding Bloch sphere state, the following function computes a dictionary of the four Stokes parameters (using the information from the 'stokes_index_to_projector_combinations_map1' map above).

If one has experimental data corresponding with the probabilities of measuring the six Bloch sphere states for a given arbitrary single qubit state, formatted into a dictionary format, the following computes the Stokes parameters. While this is somewhat trivial in the single qubit state tomography case, such mappings become useful in the two qubit state tomography case.

In [ ]:
def compute_stokes_parameters1(state_to_prob_map, should_print_description = False):

    param = 0
    assoc = {}

    func = lambda x: state_to_prob_map[x]

    for j in range(len(stokes_index_to_projector_combinations_map1)):

        param = sum(list(map(func, stokes_index_to_projector_combinations_map1[j][0]))) - sum(list(map(func, stokes_index_to_projector_combinations_map1[j][1])))

        if should_print_description:
            print(f"Stokes S{j} parameter: {param}\n")
        
        assoc[f"S{j}"] = param
    
    return assoc

Given a dictionary of Stokes parameters, the following reconstructs the inferred single qubit density matrix.

In [ ]:
def reconstruct_state1(stokes_assoc):

    state = 0.0 + 0.0*1j

    for j in range(len(stokes_assoc)):

        state += 0.5 * stokes_assoc[f"S{j}"] * stokes_index_to_pauli1[j]
    
    return state

### Single qubit examples

Single qubit state $\rho$ to perform state tomography on.

In [ ]:
rho = 0.5 * np.array([[1, -1j], [1j, 1]])

Compute the probabilities of measuring the six cardinal Bloch sphere states after repeated measurements on an ensemble of identical states $\rho$.

In [ ]:
state_probabilities = compute_probabilities1(rho, should_print_description)
state_probabilities

In the lab, we may not know what the state $\rho$ is, and "compute_probabilities1" simply simulated measurements, given this input state $\rho$ above.

In practice, for an unknown state, we simply need to form a dictionary of probabilities (which are collected experimentally), similar to the one outputted by the above "state_probabilities" variable, by taking measurements according to the printed description.

This then gets fed into the "compute_stokes_parameters1" function below.

In [ ]:
stokes_params = compute_stokes_parameters1(state_probabilities, should_print_description)
stokes_params

Use the Stokes parameters above, which were derived from various combinations of the experimental measurement probabilities in the dictionary, to reconstruct the density matrix state.

In [ ]:
reconstruct_state1(stokes_params)

### Two qubit functions

Maps an integer to a string representing one of the thirty-six cardinal points of the Bloch spheres of two qubits. Akin to 'integer_to_state_string_map1' above, now just for two qubits.

In [ ]:
integer_to_state_string_map2 = {
   1: "|0>|0>",
   2: "|0>|1>",
   3: "1/sqrt(2)|0>(|0>+|1>)",
   4: "1/sqrt(2)|0>(|0>-|1>)",
   5: "1/sqrt(2)|0>(|0>+i|1>)",
   6: "1/sqrt(2)|0>(|0>-i|1>)",
   7: "|1>|0>",
   8: "|1>|1>",
   9: "1/sqrt(2)|1>(|0>+|1>)",
   10: "1/sqrt(2)|1>(|0>-|1>)",
   11: "1/sqrt(2)|1>(|0>+i|1>)",
   12: "1/sqrt(2)|1>(|0>-i|1>)",
   13: "1/sqrt(2)(|0>+|1>)|0>",
   14: "1/sqrt(2)(|0>+|1>)|1>",
   15: "1/2(|0>+|1>)(|0>+|1>)",
   16: "1/2(|0>+|1>)(|0>-|1>)",
   17: "1/2(|0>+|1>)(|0>+i|1>)",
   18: "1/2(|0>+|1>)(|0>-i|1>)",
   19: "1/sqrt(2)(|0>-|1>)|0>",
   20: "1/sqrt(2)(|0>-|1>)|1>",
   21: "1/2(|0>-|1>)(|0>+|1>)",
   22: "1\2(|0>-|1>)(|0>-|1>)",
   23: "1\2(|0>-|1>)(|0>+i|1>)",
   24: "1\2(|0>-|1>)(|0>-i|1>)",
   25: "1/sqrt(2)(|0>+i|1>)|0>",
   26: "1/sqrt(2)(|0>+i|1>)|1>",
   27: "1/2(|0>+i|1>)(|0>+|1>)",
   28: "1/2(|0>+i|1>)(|0>-|1>)",
   29: "1/2(|0>+i|1>)(|0>+i|1>)",
   30: "1/2(|0>+i|1>)(|0>-i|1>)",
   31: "1/sqrt(2)(|0>-i|1>)|0>",
   32: "1/sqrt(2)(|0>-i|1>)|1>",
   33: "1/2(|0>-i|1>)(|0>+|1>)",
   34: "1/2(|0>-i|1>)(|0>-|1>)",
   35: "1/2(|0>-i|1>)(|0>+i|1>)",
   36: "1/2(|0>-i|1>)(|0>-i|1>)"
}

Maps an integer to a numpy array representation of the two-qubit state corresponding to the string representation from the above 'integer_to_state_string_map2' dictionary. Akin to 'integer_to_state_string_map1', now just for two qubits.

In [ ]:
integer_to_state_ket_map2 = {
    1: np.array([1, 0, 0, 0]),
    2: np.array([0, 1, 0, 0]),
    3: 1/np.sqrt(2) * np.array([1, 1, 0, 0]),
    4: 1/np.sqrt(2) * np.array([1, -1, 0, 0]),
    5: 1/np.sqrt(2) * np.array([1, 1j, 0, 0]),
    6: 1/np.sqrt(2) * np.array([1, -1j, 0, 0]),
    7: np.array([0, 0, 1, 0]),
    8: np.array([0, 0, 0, 1]),
    9: 1/np.sqrt(2) * np.array([0, 0, 1, 1]),
    10: 1/np.sqrt(2) * np.array([0, 0, 1, -1]),
    11: 1/np.sqrt(2) * np.array([0, 0, 1, 1j]),
    12: 1/np.sqrt(2) * np.array([0, 0, 1, -1j]),
    13: 1/np.sqrt(2) * np.array([1, 0, 1, 0]),
    14: 1/np.sqrt(2) * np.array([0, 1, 0, 1]),
    15: (1/2)*np.array([1, 1, 1, 1]),
    16: (1/2)*np.array([1, -1, 1, -1]),
    17: (1/2)*np.array([1, 1j, 1, 1j]),
    18: (1/2)*np.array([1, -1j, 1, -1j]),
    19: 1/np.sqrt(2) * np.array([1, 0, -1, 0]),
    20: 1/np.sqrt(2) * np.array([0, 1, 0, -1]),
    21: (1/2)*np.array([1, 1, -1, -1]),
    22: (1/2)*np.array([1, -1, -1, 1]),
    23: (1/2)*np.array([1, 1j, -1, -1j]),
    24: (1/2)*np.array([1, -1j, -1, 1j]),
    25: 1/np.sqrt(2) * np.array([1, 0, 1j, 0]),
    26: 1/np.sqrt(2) * np.array([0, 1, 0, 1j]),
    27: (1/2)*np.array([1, 1, 1j, 1j]),
    28: (1/2)*np.array([1, -1, 1j, -1j]),
    29: (1/2)*np.array([1, 1j, 1j, -1]),
    30: (1/2)*np.array([1, -1j, 1j, 1]),
    31: 1/np.sqrt(2) * np.array([1, 0, -1j, 0]),
    32: 1/np.sqrt(2) * np.array([0, 1, 0, -1j]),
    33: (1/2)*np.array([1, 1, -1j, -1j]),
    34: (1/2)*np.array([1, -1, -1j, 1j]),
    35: (1/2)*np.array([1, 1j, -1j, 1]),
    36: (1/2)*np.array([1, -1j, -1j, -1])
}

Maps indicating the ideal separable two-qubit gate that needs to be applied to qubits initialised in their ground states, in order to create the state with the string representation from the 'integer_to_state_string_map2' map above, or the numpy array representation from the 'integer_to_state_ket_map2' map above. Akin to 'make_state_from_ground_map1', now just for two qubits

In [ ]:
make_state_from_ground_map2 = {
    1: "I⊗I",
    2: "I⊗(X180/Y180)",
    3: "I⊗Y90",
    4: "I⊗Y-90",
    5: "I⊗X-90",
    6: "I⊗X90",
    7: "(X180/Y180)⊗I",
    8: "(X180/Y180)⊗(X180/Y180)",
    9: "(X180/Y180)⊗Y90",
    10: "(X180/Y180)⊗Y-90",
    11: "(X180/Y180)⊗X-90",
    12: "(X180/Y180)⊗X90",
    13: "Y90⊗I",
    14: "Y90⊗(X180/Y180)",
    15: "Y90⊗Y90",
    16: "Y90⊗Y-90",
    17: "Y90⊗X-90",
    18: "Y90⊗X90",
    19: "Y-90⊗I",
    20: "Y-90⊗(X180/Y180)",
    21: "Y-90⊗Y90",
    22: "Y-90⊗Y-90",
    23: "Y-90⊗X-90",
    24: "Y-90⊗X90",
    25: "X-90⊗I",
    26: "X-90⊗(X180/Y180)",
    27: "X-90⊗Y90",
    28: "X-90⊗Y-90",
    29: "X-90⊗X-90",
    30: "X-90⊗X90",
    31: "X90⊗I",
    32: "X90⊗(X180/Y180)",
    33: "X90⊗Y90",
    34: "X90⊗Y-90",
    35: "X90⊗X-90",
    36: "X90⊗X90"
}

Maps the stokes index ($i$ in $S_{i}$, where $S_{i}$ is a Stokes parameter) to the Pauli operator basis decomposisiton of a two-qubit density matrix. Akin to 'stokes_index_to_pauli1', now just for two qubits.

In [ ]:
stokes_index_to_pauli2 = {
    0: kron(i,i),
    1: kron(i,x),
    2: kron(i,y),
    3: kron(i,z),
    4: kron(x,i),
    5: kron(x,x),
    6: kron(x,y),
    7: kron(x,z),
    8: kron(y,i),
    9: kron(y,x),
    10: kron(y,y),
    11: kron(y,z),
    12: kron(z,i),
    13: kron(z,x),
    14: kron(z,y),
    15: kron(z,z)
}

A dictionary which maps the Stokes parameter index to a list of two lists. The first is a list of integers corresponding to separable two-qubit Bloch sphere states (as in the 'integer_to_state_string_map2' map above), projective measurement values of which need to be ADDED together, while the second is a list of integers corresponding to two qubit Bloch sphere states, projective measurement values of which need to be SUBTRACTED, in order to reconstruct the Stokes parameter.

For example, given an arbitrary unknown density matrix for a two qubit states, to deduce the Stokes parameter $S_{2}$, we would need to measure the probability of measuring the two qubit Bloch sphere state 5, i.e. $\frac{1}{\sqrt{2}} \left |0 \right > \left ( \left | 0 \right > +i \left | 1 \right > \right )$ (giving $P_5$) plus the probability of measuring the two qubit Bloch sphere state 11, i.e. $\frac{1}{\sqrt{2}} \left |1 \right > \left ( \left | 0 \right > +i \left | 1 \right > \right )$ (giving $P_{11}$), minus the probability of measuring the two qubit Bloch sphere state 6, i.e. $\frac{1}{\sqrt{2}} \left |0 \right > \left ( \left | 0 \right > -i \left | 1 \right > \right )$ (giving $P_{6}$), minus the probability of measuring the two qubit Bloch sphere state 12, i.e. $\frac{1}{\sqrt{2}} \left |1 \right > \left ( \left | 0 \right > -i \left | 1 \right > \right )$ (giving $P_{12}$). Then $S_{2} = P_5 + P_{11} - P_{6} - P_{12}$.

Akin to 'stokes_index_to_projector_combinations_map2', now just for two qubits.

In [ ]:
stokes_index_to_projector_combinations_map2 = {
    0: [[1, 2, 7, 8], []],
    1: [[3, 9], [4, 10]],
    2: [[5, 11], [6, 12]],
    3: [[1, 7], [2, 8]],
    4: [[13, 14], [19, 20]],
    5: [[15, 22], [16, 21]],
    6: [[17, 24], [18, 23]],
    7: [[13, 20], [14, 19]],
    8: [[25, 26], [31, 32]],
    9: [[27, 34], [28, 33]],
    10: [[29, 36], [30, 35]],
    11: [[25, 32], [26, 31]],
    12: [[1, 2], [7, 8]],
    13: [[3, 10], [4, 9]],
    14: [[5, 12], [6, 11]],
    15: [[1, 8], [2, 7]]
}

Given a two qubit qubit density matrix, the following function decomposes the state into the Pauli basis and outputs a dictionary which maps the Stokes parameter index to the corresponding value.

This function is used in simulations and can be used in sanity checks.

Akin to 'compute_probabilities1', now just for two qubits.

In [ ]:
def compute_probabilities2(rho, should_print_description = False):

    assoc = {}

    if should_print_description:
        print(f"For starting state rho = {rho}:\n")
    
    for j in range(1, len(integer_to_state_string_map2) + 1):
        answer = np.round(np.trace(rho @ ketbradiag(integer_to_state_ket_map2[j])), decimals=7)

        if not np.isclose(answer.imag, 0.0):
            raise ValueError(f"The probability value {anwser} should have an imaginary part of 0.")
        else:
            answer = answer.real

        if should_print_description:
            print(f"{j}: Probability of measuring state {integer_to_state_string_map2[j]} (which is made using gate {make_state_from_ground_map2[j]} from |0>): {answer}\n")
        
        assoc[j] = answer
    
    return assoc        

Given an input dictionary with keys corresponding to the integers of the thirty-six two-qubit cardinal Bloch sphere states (as in the dictionary 'integer_to_state_string_map2') and the values corresponding with the probability of measuring the corresponding Bloch sphere states, the following function computes a dictionary of the 16 Stokes parameters (using the information from the 'stokes_index_to_projector_combinations_map2' map above).

If one has experimental data corresponding with the probabilities of measuring the thirty-six Bloch sphere states for a given arbitrary two-qubit state, formatted into a dictionary format, the following computes the Stokes parameters.

Akin to 'compute_stokes_parameters1'.

In [ ]:
def compute_stokes_parameters2(state_to_prob_map, should_print_description = False):

    param = 0
    assoc = {}

    func = lambda x: state_to_prob_map[x]

    for j in range(len(stokes_index_to_projector_combinations_map2)):

        param = sum(list(map(func, stokes_index_to_projector_combinations_map2[j][0]))) - sum(list(map(func, stokes_index_to_projector_combinations_map2[j][1])))

        if should_print_description:
            print(f"Stokes S{j} parameter: {param}\n")
        
        assoc[f"S{j}"] = param
    
    return assoc

Given a dictionary of Stokes parameters, the following reconstructs the inferred two-qubit density matrix.

In [ ]:
def reconstruct_state2(stokes_assoc):

    state = 0.0 + 0.0*1j

    for j in range(len(stokes_assoc)):

        state += 0.25 * stokes_assoc[f"S{j}"] * stokes_index_to_pauli2[j]
    
    return state

### Two qubit example

Two qubit state $\rho$ to perform state tomography on.

In [ ]:
rho = np.array([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

Compute the probabilities of measuring the thirty-six cardinal Bloch sphere states after repeated measurements on an ensemble of identical states $\rho$.

In [ ]:
state_probabilities = compute_probabilities2(rho, should_print_description)
state_probabilities

In the lab, we may not know what the state $\rho$ is, so "compute_probabilities2" simply simulated measurements, given this input state $\rho$ above.

In practice, for an unknown state, we simply need to form a dictionary of probabilities (which are collected experimentally), similar to the one outputted by the above "state_probabilities" variable, by taking measurements according to the printed description.

This then gets fed into the "compute_stokes_parameters2" function below.

In [ ]:
stokes_params = compute_stokes_parameters2(state_probabilities, should_print_description)
stokes_params

Use the Stokes parameters above, which were derived from various combinations of the experimental measurement probabilities in the dictionary, to reconstruct the density matrix state.

In [ ]:
reconstruct_state2(stokes_params)

## Quantum Process Tomography

The following cells discuss standard quantum process tomography; Refs 2 and 3 were consulted when the theory was worked out for these sections.

### Standard Process Tomography Setup

In the $\chi$ process matrix formalism, the dynamics of any open quantum system's density matrix $\rho$, under the action of a CP linear map $\varepsilon$ (the 'process'), is given by:

\begin{equation} \tag{1}
\varepsilon(\rho) = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} E_m \rho E_n^{\dagger},
\end{equation}

where $\{ E_m \}_{m=0}^{d^2 - 1}$ is an orthogonal Hermitian operator basis (satisfying $\text{Tr}(E_m^{\dagger}E_n) = d\delta_{mn}$ and $E_m = E_m^{\dagger}$), and $d=2^{N}$ with $N$ the number of qubits. From here onwards, we choose the basis to be that of the Pauli basis, $\{ I, X, Y, Z \}^{\otimes N}$. Note that this $\chi$ process matrix formalism is only one way of describing an open quantum system's dynamics; the interested reader should consult Ref 3 for other formulations.

The matrix $\chi$ completely characterises the process with respect to the basis (here the Pauli basis). For example, if the process is completely unitary, $\chi$ is generally rank 1 and $\varepsilon$ reduces to $\varepsilon(\rho) = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} E_m \rho E_n^{\dagger} \to U \rho U^{\dagger}$ for some unitary $U$. However, $\varepsilon$ is, in general, more general; when one implements an operation (which is ideally unitary in theory) on a real-world device, the operation and device itself are also subject to coherent noise (such as imperfections in pulse calibrations) and incoherent noise (qubit decay and dephasing, for example). $\varepsilon$ encompasses the unitary operation as well as these sources of noise.

Process tomography concerns itself with determining the matrix $\chi$. To do so, we need to prepare a set of tomographically complete input states, subject them to the process, and then perform a tomographically complete set of measurements. In our case, we consider standard quantum process tomography, where an over-complete set of measurements are taken, but with the mathematics being arguably easier to follow. We choose the set of tomographically complete input "states" to be the Pauli operators $\{ E_n \}$ and subject them all to the process by taking ensembles of measurements, so in other words, we prepare $\varepsilon(E_s) = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} E_m E_s E_n^{\dagger}$ for $s \in \{ 0, 1, \cdots, d^2 - 1 \}$. Furthermore, we also measure the same set, with these $E_t$ playing the role of "observables". So, we measure $E_t$, which for an input "state" $E_s$, gives $m_{st}$, where

\begin{equation} \tag{2}
m_{st} = \text{Tr}(E^{\dagger}_t\varepsilon(E_s)) = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} \text{Tr}(E^{\dagger}_t E_m E_s E_n^{\dagger}).
\end{equation}

Note that $\text{Tr}(E^{\dagger}_t E_m E_s E_n^{\dagger})$ is a constant that clearly doesn't depend on experimental data but only the chosen basis (which, as mentioned, for both the input state and the measurement observable, is the Pauli basis); $P_{stmn} = \text{Tr}(E^{\dagger}_t E_m E_s E^{\dagger}_n)$, for $s, t, m, n \in \{ 0, 1, \cdots, d^2 - 1 \}$.

However, we cannot truly prepare the states $\{ E_m \}$ in the lab since they are not true qubit(s) quantum states (for example, $\text{Tr}(E_m) = 0$ when $E_m$ is a Pauli operator that's not the identity). The set $\{ E_m \}$ are also not true projectors, since $E_m^2 \neq E_m$ in general. However, what is possible in the lab, and what is generally done, is we prepare one of the six(thirty-six) cardinal Bloch sphere states from qubit(s) initialised in their ground state(s). This state is then subject to the process represented by $\chi$, and finally, we measure the probability of the qubit(s) being in one of these six(thirty-six) Bloch sphere states by changing the measurement basis right before measuring in the qubit(s) $Z$ bases. Let $\mathcal{B} = \{ I, X_{180}, Y_{90}, Y_{-90}, X_{-90}, X_{90} \}$ be the set of six "Bloch sphere" operators which are used to prepare one of the six cardinal Bloch sphere states, and also change the measurement basis. We denote one of these six gates as $\mathcal{B}_i$, where $i \in \{ 0, 1, 2, 3, 4, 5 \}$.

### Tomography preamble

Returns one of the six operators used to create the six cardinal points of the Bloch sphere from a qubit in the ground state (corresponds to $\mathcal{B}_i$ in the above description).

In [ ]:
def func_B(n: int):
    
    match n:
        case 0: # identity
            return i
        case 1: # X180/Y180
            return x
        case 2: # Y90
            return y90
        case 3: # Y-90
            return ym90
        case 4: # X-90
            return xm90
        case 5: # X90
            return x90
        case _: # Error
            raise ValueError("Input integer should be between 0 and 5, inclusive")

Maps an integer index to either the identity or a Pauli operator.

In [ ]:
index_to_pauli_string_process1 = {
    0: "I",
    1: "X",
    2: "Y",
    3: "Z"
}

Helper function used in 'describe_pauli_state_creation_from_ground_state_process1'; maps Bloch sphere state integer to a list, where the first element of the list is a string representing the Bloch sphere rotation operator needed to create the Bloch sphere state from $\left | 0 \right >$, and the second element the matrix representation of that operator

In [ ]:
make_state_from_ground_map_process1 = {
    0: ["I", func_B(0)],
    1: ["X180/Y180", func_B(1)],
    2: ["Y90", func_B(2)],
    3: ["Y-90", func_B(3)],
    4: ["X-90", func_B(4)],
    5: ["X90", func_B(5)],
}

Helper function used in 'describe_pauli_state_creation_from_ground_state_process1'; maps an index corresponding to the Pauli operator to a list, where each element of the list is a list whereby the first element the coefficient, and the second element the Bloch sphere operator index needed to create the corresponding Bloch sphere state from the qubit ground state. Elements of the list are summed over, with the appropriate coefficient, to create the Pauli operator from combinations of Bloch sphere states

In [ ]:
index_to_from_ground_state_combinations_process1 = {
    0: [[1,0], [1,1]],
    1: [[-(1 + 1j), 0], [-(1 + 1j), 1], [2, 2], [1j, 4], [1j, 5]],
    2: [[1, 4], [-1, 5]],
    3: [[1, 0], [-1, 1]]
}

Helper function that simply describes how to create the Pauli input state from combinations of Bloch sphere states (using the above helper functions)

In [ ]:
def describe_pauli_state_creation_from_ground_state_process1(should_print_description):

    if should_print_description:
        print(f"For a single qubit, to create a Pauli input state from linear combinations of Bloch sphere operators acting on the ground state |0>:\n")
    
    for k in range(len(index_to_from_ground_state_combinations_process1)):

        if should_print_description:

            temp_string = ""
            temp_list = index_to_from_ground_state_combinations_process1[k]

            for j in range(len(temp_list)):
                temp_string = temp_string + str(temp_list[j][0]) + " x " + str(make_state_from_ground_map_process1[temp_list[j][1]][0]) + "|0><0|(" + str(make_state_from_ground_map_process1[temp_list[j][1]][0]) + ")^† + "
            
            temp_string = temp_string[:-3]

            print(f"{k}: {index_to_pauli_string_process1[k]} = {temp_string}")


In [ ]:
describe_pauli_state_creation_from_ground_state_process1(True)

### Single qubit functions

#### Single qubit case

For a single qubit, preparing Bloch state $\mathcal{B}_i \left | 0 \right >$, subjecting the qubit to the process represented by $\chi$, changing basis using $\mathcal{B}_j$ and then measuring the probability of finding the qubit in the ground state $\left | 0 \right >$, gives the probability $\lambda_{ij}$, where

\begin{equation} \tag{3}
\lambda_{ij} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} \text{Tr}( \mathcal{B}^{\dagger}_j\left | 0 \right > \left < 0 \right | \mathcal{B}_j E_m \mathcal{B}_i\left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i E^{\dagger}_n),
\end{equation}

for $i, j \in \{ 0, 1, 2, 3, 4, 5  \}$. Note that $\text{Tr}( \mathcal{B}^{\dagger}_j\left | 0 \right > \left < 0 \right | \mathcal{B}_j E_m \mathcal{B}_i\left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i E^{\dagger}_n)$ is also a constant independent of any experimental data. We represent it as $B_{ijmn} = \text{Tr}( \mathcal{B}^{\dagger}_j\left | 0 \right > \left < 0 \right | \mathcal{B}_j E_m \mathcal{B}_i\left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i E^{\dagger}_n)$.

These measurements $\lambda_{ij}$, in terms of the Bloch sphere states, are what we observe in the lab. However, we have chosen to describe the $\chi$ matrix formalism in terms of the Pauli basis. Thankfully, we can exploit the linearity of the $\varepsilon$ process and map results from the Bloch state case to the Pauli basis by taking linear combinations of Bloch sphere state measurements. In other words, we need to express each Pauli matrix $E_m$ as a linear combination of Bloch sphere states $\mathcal{B}_i\left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i$. This is simple enough to do; the coefficients $c_{mi}$, are such that

\begin{equation} \tag{4}
E_m = \sum_{i=0}^5 c_{mi} \mathcal{B}_i \left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i,
\end{equation}

are unique and listed below. Using this, we can map $B_{ijmn}$ to $P_{stmn}$ as

\begin{equation} \tag{5}
P_{stmn} = \sum_{i, j=0}^5 c_{si} c^{*}_{tj} B_{ijmn}.
\end{equation}

There is also a mapping between Bloch state measurement data, $\lambda_{ij}$, to Pauli state measurements, $m_{st}$, via

\begin{equation} \tag{6}
m_{st} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} P_{stmn} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} \sum_{i, j=0}^5 c_{si} c^{*}_{tj} B_{ijmn} = \sum_{i, j=0}^5 c_{si} c^{*}_{tj} \lambda_{ij}
\end{equation}


We have all of the logic needed to perform single qubit process tomography:

1. Collect measurement data $\lambda_{ij}$ from the lab corresponding to preparing Bloch state $i$ and measuring Bloch state $j$
2. Map this data to the Pauli basis $m_{st}$ using Eq. 6
3. Solve Eq. 2, i.e. $m_{st} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} P_{stmn}$, for $\chi$

Below, `func_E(i)` corresponds with $E_i$, and `func_c(s, i)` with $c_{si}$

In [ ]:
def func_E(n: int):
    # four single qubit Pauli operators
    match n:
        case 0: # identity
            return i
        case 1: # X
            return x
        case 2: # Y
            return y
        case 3: # Z
            return z
        case _:
            raise ValueError("Input integer should be between 0 and 3, inclusive")

@functools.lru_cache()
def func_c(s: int, i: int):
    # Constants func_c(s,i) such that
    # func_E(s) = sum_{i=0}^5 func_c(s,i) func_B(i)|0><0|func_B^{\dagger}(i)
    match (s, i):
        case (0,0):
            return 1
        case (0,1):
            return 1
        case (1,0):
            return -(1 + 1j)
        case (1,1):
            return -(1 + 1j)
        case (1,2):
            return 2
        case (1,4):
            return 1j
        case (1,5):
            return 1j
        case (2,4):
            return 1
        case (2,5):
            return -1
        case (3,0):
            return 1
        case (3,1):
            return -1
        case _:
            return 0

For any single qubit input state $\rho$, the following decomposes $\rho$ into the Pauli operator basis $E_i$

In [ ]:
def decompose_state_into_pauli_basis_process1(rho, should_print_description = False):

    if not ishermitian(rho):
        print(f"Matrix rho is not Hermitian")
    else:
        if not is_pos_semidef(rho):
            print(f"Matrix rho is not positive semi-definite")
    if not np.isclose(np.trace(rho), 1.0):
        print("Matrix rho is not normalised")
    
    coefficients = []

    if should_print_description:
        print("Input state rho decomposed in the Pauli basis E(i), i.e. rho = \sum a(i)E(i) is:\n")
    
    for j in range(len(rho[0])**2):
        result = 0.5 * np.trace((func_E(j).conj().T) @ rho)

        if not np.isclose(result.imag, 0.0):
            raise ValueError(f"Result {result} should have no imaginary part")
        else:
            result = result.real

        coefficients.append(result)

        if should_print_description:
            print(f"{j}) a({j}) = 1/2 * Tr[E({j}).rho] = {result}")
    
    return coefficients

Test for $\rho = \left | 0 \right > \left < 0 \right |$

In [ ]:
rho_test = np.array([[1,0],[0,0]])

decompose_state_into_pauli_basis_process1(rho_test, True)

For an input $j$, describes decomposing an input Pauli operator $E_j$ into a linear combination of single qubit Bloch sphere states $\mathcal{B}_i \left | 0 \right > \left < 0 \right | \mathcal{B}^{\dagger}_i$

In [ ]:
def describe_decomposing_pauli_operator_into_rank_1_bloch_state1(j, should_print_description):

    if should_print_description:
        print(f"Input Pauli operator E({j}) decomposed into the rank 1 Bloch sphere states, i.e. E({j}) = \sum c({j},i)F(i)|0><0|F(i)^dagger, is:\n")
    
    for k in range(6):

        if should_print_description:
            print(f"{k}) Coefficient of F({k}) term is: c({j},{k}) = {func_c(j, k)}\n")

In [ ]:
describe_decomposing_pauli_operator_into_rank_1_bloch_state1(1, True)

Helper function to check the math

In [ ]:
def reconstruct_pauli_from_rank_1_bloch_states1(j):

    answer = 0.0 + 0.0*1j

    for k in range(6):
        answer += func_c(j, k) * (func_B(k) @ np.array([[1,0],[0,0]]) @ (func_B(k).conj().T))
    
    return answer

def check_EC_mapping(j):

    reconstructed_operator = reconstruct_pauli_from_rank_1_bloch_states1(j)

    if not np.allclose(reconstructed_operator, func_E(j)):

        result = False

        print(f"Reconstruction for operator E({j}) failed!\n")
    
    else:
        result = True

        print(f"Reconstruction for operator E({j}) passed!\n")
    
    return result

for j in range(4):

    check_EC_mapping(j)

Corresponds to $B_{ijmn}$ in Eq. 3 above

In [ ]:
def B_Bloch(i, j, m, n):

    # start from qubit ground state, create one of six Bloch sphere states
    # using func_B, add Pauli operator basis states func_E, and measure one
    # of the six Bloch sphere state projectors,
    # func_B(.)^{\dagger} |0><0| func_B(.). These constants appear when expressing
    # the Pauli states in terms of combinations of Bloch sphere states,
    # assuming you start from the qubit ground state

    starting_state = np.array([[1,0],[0,0]])
    bloch_state = func_B(i) @ starting_state @ (func_B(i).conj().T)
    add_paulis = func_E(m) @ bloch_state @ (func_E(n).conj().T)
    measure_bloch_state = (func_B(j).conj().T) @ starting_state @ func_B(j) @ add_paulis

    return measure_bloch_state.trace()

Corresponds to $P_{stmn}$ in Eq. 2 above, and explicitly performs the mapping of Eq. 5

In [ ]:
def P_Pauli(s, t, m, n):

    result = 0

    for i in range(0, 6):
        for j in range(0, 6):
            result += func_c(s, i) * np.conj(func_c(t, j)) * B_Bloch(i, j, m, n)

    return result.round(8)

Prints out a description of Eq 5 per index

In [ ]:
def convert_Matrix_from_bloch_to_pauli(should_print_description):

    if should_print_description:

        print(f"The P matrix, converted from the B matrix (i.e. from Bloch states to Paulis), is:\n")
    
        for k in range(4):
            for l in range(4):
                for m in range(4):
                    for n in range(4):
                        print(f"The matrix element Tr[E^dagger({k})E({m})E({l})E^dagger({n})] is: P({l},{k},{m},{n}) = {P_Pauli(l, k, m, n)}")

In [ ]:
convert_Matrix_from_bloch_to_pauli(True)

Reshapes $P_{stmn}$ into a 2D matrix (used to solve Eq. 2)

In [ ]:
PMatrix = np.array([[P_Pauli(np.floor(v/4).astype(int), v % 4, np.floor(w/4).astype(int), w % 4) for w in range(16)] for v in range(16)]).round(8)

### Single qubit example: amplitude damping

Simulate the amplitude damping noise channel on a single qubit, with probability to damp, $p$, equal to 13% (see Ref 3)

In [ ]:
p = 0.13

k0 = np.array([[1, 0],[0, np.sqrt(1-p)]])
k1 = np.array([[0, np.sqrt(p)],[0, 0]])

The damping process itself, in terms of Kraus operators

In [ ]:
def damping(rho):

    return (k0 @ rho @ (k0.conj().T)) + (k1 @ rho @ (k1.conj().T))

Initialise empty Bloch sphere basis measurement array 'lamb' (corresponding to $\lambda$), and Pauli basis array 'm' (corresponding to $m$)

In [ ]:
m = np.empty((4, 4))
lamb = np.empty((6, 6))

Simulate the process of performing single qubit process tomography for the input 'process', by preparing and measuring the complete set of Bloch sphere states

In [ ]:
def simulate_process_tomography_measurements_with_input_bloch_states_and_measuring_bloch_projectors(process, should_print_description):

    for i in range(6):
        for j in range(6):

            rho = np.array([[1, 0], [0, 0]])

            rho = func_B(i) @ rho @ (func_B(i).conj().T)

            rho = process(rho)

            rho = (func_B(j).conj().T) @ np.array([[1, 0], [0, 0]]) @ func_B(j) @ rho

            res = np.trace(rho)

            if not np.isclose(res.imag, 0.0):
                raise ValueError(f"Result {res} should have no imaginary part")
            else:
                res = res.real

            lamb[i, j] = res

            if should_print_description:

                print(f"Measurement result of input process for the input state F({i})|0> and Bloch projector F({j})|0><0|Fdag({j}) = {lamb[i, j]}\n")

In [ ]:
simulate_process_tomography_measurements_with_input_bloch_states_and_measuring_bloch_projectors(damping, True)

For a given input Bloch sphere measuremenet matrix (here called 'lamb'), describes the corresponding Pauli measurement matrix (here 'm')

In [ ]:
def map_from_bloch_state_measurements_to_pauli(arr, should_print_description):

    if arr.shape != (6, 6):
        raise ValueError("Input array must be 6x6")
    
    if should_print_description:
        print(f"For the Bloch states' measurements, the Pauli basis m measurement matrix's elements are:\n")

    for k in range(4):
        for l in range(4):

            result = 0
    
            for i in range(0, 6):
                for j in range(0, 6):
                    result += func_c(l, i) * np.conj(func_c(k, j)) * arr[i][j]
            
            if not np.isclose(result.imag, 0.0):
                raise ValueError(f"Result {result} should have no imaginary part")
            else:
                result = result.real
            
            m[l, k] = result
            
            if should_print_description:
                print(f"If the Pauli state E({l}) were prepared and the Pauli observable E({k}) measured, the measured value would be: Tr(E^(dagger)({k})ε(E({l}))) = m({l},{k}) = {result}")

In [ ]:
map_from_bloch_state_measurements_to_pauli(lamb, True)

Reshapes Pauli measurement matrix 'm' into a 1D vector.

In [ ]:
measurement_vector = np.array([m[np.floor(v/4).astype(int), v % 4] for v in range(16)])

Solves Eq. 2 for $\chi$ in a vectorised form, and reshapes the vector into the $\chi_{mn}$ matrix

In [ ]:
chi_vector = solve(PMatrix, measurement_vector)
chi_matrix = chi_vector.reshape(4,4)

Plot the $\chi$ matrix

In [ ]:
def plot_process_tomography(chi_vector, save_file: str = None):

    plt.rcParams['text.usetex'] = True

    cmap = colormaps.get_cmap('viridis')

    # set up figure
    fig = plt.figure(figsize=(9, 3), dpi=250, facecolor="white")
    ax = fig.add_subplot(111, projection='3d')

    # coordinates
    r = np.arange(4)
    _x, _y = np.meshgrid(r, r)
    x, y = _x.ravel(), _y.ravel()

    values0 = np.abs(chi_vector)
    values1 = np.angle(chi_vector)

    top = values0
    bottom = np.zeros_like(top)

    width = depth = 0.7

    norm = Normalize(vmin=-np.pi, vmax=np.pi)
    colors = cmap(norm(values1))

    xy_ticks_labels = [r"$I$", r"$X$", r"$Y$", r"$Z$"]

    subplot = ax.bar3d(x, y, bottom, width, depth, top, shade=True, color=colors)
    ax.set_xticks(r + 0.5, labels=xy_ticks_labels)
    ax.set_yticks(r + 0.5, labels=xy_ticks_labels)
    ax.set_zticks([0, (max(top)/2).round(2), max(top).round(2)])
    ax.set_xlabel("Prepared", fontdict = axisfont)
    ax.set_ylabel("Measured", fontdict = axisfont)
    ax.set_title(r'$\chi$ matrix', fontdict = titlefont)
    ax.view_init(20, -60, 0)

    sc = cm.ScalarMappable(cmap=cmap,norm=norm)
    cbar = plt.colorbar(sc, ax=ax, pad=0.1, shrink=0.7)
    cbar.set_ticks(ticks=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi], labels=[r"$-\pi$", r"$-\pi/2$", r"0", r"$\pi/2$", r"$\pi$"])

    if save_file:
        plt.savefig(save_file, bbox_inches='tight')

    plt.show()

In [ ]:
plot_process_tomography(chi_vector)

Given the experimental $\chi$ matrix, extract the predicted amplitude damping probability value, assuming only a perfect damplitude damping noise channel (again, see Ref 3)

In [ ]:
predicted_prob = 1 - (np.sqrt(4*chi_matrix[0,0]) - 1)**2

Compare the input and inferred values

In [ ]:
np.isclose(predicted_prob, p)

Check trace-preserving-ness: if $\chi$ is trace-preserving, it must satisfy $\sum_{m,n=0}^{d^2 - 1} \chi_{mn} E_{n} E_m = I$

In [ ]:
sum_of_terms = np.sum([chi_matrix[m, n] * func_E(n) @ func_E(m) for n in range(4) for m in range(4)], axis=0)

In [ ]:
if np.allclose(sum_of_terms, np.eye(2)):
    print("Process is trace preserving")
else:
    print("Process is not trace preserving")

Given $\chi$ matrix, compute the corresponding Kraus operators

In [ ]:
def compute_kraus_operators(chi_matrix, should_print_description):

    print("NOTE: NEED TO DEBUG THIS - CHECK WHETHER chi_matrix NEEDS TO BE UNITARY OR WHETHER THIS IS VALID REGARDLESS")

    kraus_operators = []

    eigenvalues, eigenvectors = eig(chi_matrix)

    if not np.allclose(eigenvectors @ (eigenvectors.conj().T), np.eye(4)):
        print("Similarity matrix is not unitary!")
    
    for m in range(4):
        
        operator = np.sqrt(eigenvalues[m]) * np.sum([eigenvectors[j, m]*func_E(j) for j in range(4)], axis=0)

        if should_print_description and not np.allclose(operator, np.zeros((2,2))):
            print(f"Kraus operator for arbitrary index m = {m}: {operator}")
            kraus_operators.append(operator.round(6))
    
    return kraus_operators

In [ ]:
compute_kraus_operators(chi_matrix, True)

### Single qubit example: depolarisation

New example: depolarisation noise channel with probability $p = 25\%$ of having a completely mixed state (see Ref 3). See the previous example for explanations, as the only thing that changes in this example is the process itself.

NOTE: you must run the previous example first to load some of the forthcoming functions

In [ ]:
p = 0.25

k0 = np.sqrt(1-3*p/4) * i
k1 = np.sqrt(p)/2 * x
k2 = np.sqrt(p)/2 * y
k3 = np.sqrt(p)/2 * z

In [ ]:
def depolarisation(rho):

    return (k0 @ rho @ (k0.conj().T)) + (k1 @ rho @ (k1.conj().T)) + (k2 @ rho @ (k2.conj().T)) + (k3 @ rho @ (k3.conj().T))

In [ ]:
m = np.empty((4, 4))
lamb = np.empty((6, 6))

In [ ]:
simulate_process_tomography_measurements_with_input_bloch_states_and_measuring_bloch_projectors(depolarisation, True)

In [ ]:
map_from_bloch_state_measurements_to_pauli(lamb, True)

In [ ]:
measurement_vector = np.array([m[np.floor(v/4).astype(int), v % 4] for v in range(16)])

In [ ]:
chi_vector = solve(PMatrix, measurement_vector)
chi_matrix = chi_vector.reshape(4,4)

In [ ]:
plot_process_tomography(chi_vector)

Given the experimental $\chi$ matrix, extract the predicted depolarisation probability value, assuming only a perfect depolarising noise channel.

In [ ]:
predicted_prob = 4*chi_matrix[1,1]

Compare the input and inferred values

In [ ]:
np.isclose(predicted_prob, p)

Check trace-preserving-ness

In [ ]:
sum_of_terms = np.sum([chi_matrix[m, n] * func_E(n) @ func_E(m) for n in range(4) for m in range(4)], axis=0)

In [ ]:
if np.allclose(sum_of_terms, np.eye(2)):
    print("Process is trace preserving")
else:
    print("Process is not trace preserving")

In [ ]:
compute_kraus_operators(chi_matrix, True)

### Two qubit functions

#### Two qubit case

The following explanation is extremely similar to that of the single qubit case, simple generalised to two qubits now.

For two qubits, preparing a Bloch state $\mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right > \left | 0 \right >$, subjecting the qubits to the process represented by $\chi$, changing basis using $\mathcal{B}_k \otimes \mathcal{B}_l$ and then measuring the probability of finding the qubits in their ground state $\left | 0 \right >\left | 0 \right >$, gives the probability $\lambda_{ijkl}$, where

\begin{equation} \tag{7}
\lambda_{ijkl} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} \text{Tr}( \mathcal{B}^{\dagger}_k \otimes \mathcal{B}^{\dagger}_l \left | 0 \right > \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}_k \otimes \mathcal{B}_l E_m \mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right >  \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j E^{\dagger}_n),
\end{equation}

for $i, j, k, l \in \{ 0, 1, 2, 3, 4, 5  \}$. Note that $\text{Tr}( \mathcal{B}^{\dagger}_k \otimes \mathcal{B}^{\dagger}_l \left | 0 \right > \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}_k \otimes \mathcal{B}_l E_m \mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right >  \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j E^{\dagger}_n)$ is also a constant independent of any experimental data. We represent it as $B_{ijklmn} = \text{Tr}( \mathcal{B}^{\dagger}_k \otimes \mathcal{B}^{\dagger}_l \left | 0 \right > \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}_k \otimes \mathcal{B}_l E_m \mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right >  \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j E^{\dagger}_n)$.

These measurements $\lambda_{ijkl}$, in terms of the Bloch sphere states, are what we observe in the lab. However, we have chosen to describe the $\chi$ matrix formalism in terms of the Pauli basis. Thankfully, we can exploit the linearity of the $\varepsilon$ process and map results from the Bloch state case to the Pauli basis by taking linear combinations of Bloch sphere state measurements. In other words, we need to express each (two-qubit) Pauli matrix $E_m$ (where $m \in \{ 0, 1, \cdots, 15 \}$) as a linear combination of Bloch sphere states $\mathcal{B}_i  \otimes \mathcal{B}_j \left | 0 \right >  \left | 0 \right >\left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j$. This is simple enough to do; the coefficients $c_{mij}$, are such that

\begin{equation} \tag{8}
E_m = \sum_{i,j=0}^5 c_{mij} \mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right > \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j,
\end{equation}

are unique and listed below. Using this, we can map $B_{ijklmn}$ to $P_{stmn}$ as

\begin{equation} \tag{9}
P_{stmn} = \sum_{i,j,k,l=0}^5 c_{sij} c^{*}_{tkl} B_{ijklmn}.
\end{equation}

There is also a mapping between Bloch state measurement data, $\lambda_{ijkl}$, to Pauli state measurements, $m_{st}$, via

\begin{equation} \tag{10}
m_{st} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} P_{stmn} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} \sum_{i, j, k, l=0}^5 c_{sij} c^{*}_{tkl} B_{ijklmn} = \sum_{i,j,k,l=0}^5 c_{sij} c^{*}_{tkl} \lambda_{ijkl}
\end{equation}

We have all of the logic needed to perform process tomography for two qubits:

1. Collect measurement data $\lambda_{ijkl}$ from the lab corresponding to preparing Bloch state $i$ on qubit 1 and $j$ on qubit 2, and measuring Bloch state $k$ on qubit 1 and $l$ on qubit 2
2. Map this data to the Pauli basis $m_{st}$ using Eq. 10
3. Solve Eq. 2, i.e. $m_{st} = \sum_{m,n = 0}^{d^2 - 1} \chi_{mn} P_{stmn}$, for $\chi$

Below, `func_E2(i)` corresponds with $E_i$, and `func_c2(s, i, j)` with $c_{sij}$, for this two-qubit case

In [ ]:
def func_E2(n: int):
    # 16 two-qubit Pauli operators
    
    i = np.array([[1,0],[0,1]])
    x = np.array([[0,1],[1,0]])
    y = np.array([[0,-1j],[1j,0]])
    z = np.array([[1,0],[0,-1]])

    match n:
        case 0: # I x I
            return np.kron(i, i)
        case 1: # I x X
            return np.kron(i, x)
        case 2: # I x Y
            return np.kron(i, y)
        case 3: # I x Z
            return np.kron(i, z)
        case 4: # X x I
            return np.kron(x, i)
        case 5: # X x X
            return np.kron(x, x)
        case 6: # X x Y
            return np.kron(x, y)
        case 7: # X x Z
            return np.kron(x, z)
        case 8: # Y x I
            return np.kron(y, i)
        case 9: # Y x X
            return np.kron(y, x)
        case 10: # Y x Y
            return np.kron(y, y)
        case 11: # Y x Z
            return np.kron(y, z)
        case 12: # Z x I
            return np.kron(z, i)
        case 13: # Z x X
            return np.kron(z, x)
        case 14: # Z x Y
            return np.kron(z, y)
        case 15: # Z x Z
            return np.kron(z, z)
        case _:
            raise ValueError("Input integer should be between 0 and 15, inclusive")

@functools.lru_cache()
def func_c2(i: int, j: int, k: int):
    # Constants func_c2(i,j,k) such that
    # func_E2(i) = sum_{j,k} func_c2(i,j,k) (func_B(j) x func_B(k)) |0>|0><0|<0| (func_B(j)^{\dagger} x func_B(k)^{\dagger})
    match (i, j, k):
        case (0,0,0):
            return 1
        case (0,0,1):
            return 1
        case (0,1,0):
            return 1
        case (0,1,1):
            return 1
        case (1,0,0):
            return -(1 + 1j)
        case (1,0,1):
            return -(1 + 1j)
        case (1,0,2):
            return 2
        case (1,0,4):
            return 1j
        case (1,0,5):
            return 1j
        case (1,1,0):
            return -(1 + 1j)
        case (1,1,1):
            return -(1 + 1j)
        case (1,1,2):
            return 2
        case (1,1,4):
            return 1j
        case (1,1,5):
            return 1j
        case (2,0,4):
            return 1
        case (2,0,5):
            return -1
        case (2,1,4):
            return 1
        case (2,1,5):
            return -1
        case (3,0,0):
            return 1
        case (3,0,1):
            return -1
        case (3,1,0):
            return 1
        case (3,1,1):
            return -1
        case (4,0,0):
            return -(1 + 1j)
        case (4,0,1):
            return -(1 + 1j)
        case (4,1,0):
            return -(1 + 1j)
        case (4,1,1):
            return -(1 + 1j)
        case (4,2,0):
            return 2
        case (4,2,1):
            return 2
        case (4,4,0):
            return 1j
        case (4,4,1):
            return 1j
        case (4,5,0):
            return 1j
        case (4,5,1):
            return 1j
        case (5,0,0):
            return (1 + 1j)**2
        case (5,0,1):
            return (1 + 1j)**2
        case (5,1,0):
            return (1 + 1j)**2
        case (5,1,1):
            return (1 + 1j)**2
        case (5,0,2):
            return -2 * (1 + 1j)
        case (5,1,2):
            return -2 * (1 + 1j)
        case (5,2,0):
            return -2 * (1 + 1j)
        case (5,2,1): 
            return -2 * (1 + 1j)
        case (5,0,4):
            return -1j * (1 + 1j)
        case (5,0,5):
            return -1j * (1 + 1j)
        case (5,1,4):
            return -1j * (1 + 1j)
        case (5,1,5):
            return -1j * (1 + 1j)
        case (5,4,0):
            return -1j * (1 + 1j)
        case (5,4,1):
            return -1j * (1 + 1j)
        case (5,5,0):
            return -1j * (1 + 1j)
        case (5,5,1):
            return -1j * (1 + 1j)
        case (5,2,2):
            return 4
        case (5,2,4):
            return 2*1j
        case (5,2,5):
            return 2*1j
        case (5,4,2):
            return 2*1j
        case (5,5,2):
            return 2*1j
        case (5,4,4):
            return -1
        case (5,4,5):
            return -1
        case (5,5,4):
            return -1
        case (5,5,5):
            return -1
        case (6,0,4):
            return -(1 + 1j)
        case (6,1,4):
            return -(1 + 1j)
        case (6,0,5):
            return (1 + 1j)
        case (6,1,5):
            return (1 + 1j)
        case (6,2,4):
            return 2
        case (6,2,5):
            return -2
        case (6,4,4):
            return 1j
        case (6,5,4):
            return 1j
        case (6,4,5):
            return -1j
        case (6,5,5):
            return -1j
        case (7,0,0):
            return -(1 + 1j)
        case (7,1,0):
            return -(1 + 1j)
        case (7,0,1):
            return (1 + 1j)
        case (7,1,1):
            return (1 + 1j)
        case (7,2,0):
            return 2
        case (7,2,1):
            return -2
        case (7,4,0):
            return 1j
        case (7,5,0):
            return 1j
        case (7,4,1):
            return -1j
        case (7,5,1):
            return -1j
        case (8,4,0):
            return 1
        case (8,4,1):
            return 1
        case (8,5,0):
            return -1
        case (8,5,1):
            return -1
        case (9,4,0):
            return -(1 + 1j)
        case (9,4,1):
            return -(1 + 1j)
        case (9,5,0):
            return (1 + 1j)
        case (9,5,1):
            return (1 + 1j)
        case (9,4,4):
            return 1j
        case (9,4,5):
            return 1j
        case (9,5,4):
            return -1j
        case (9,5,5):
            return -1j
        case (9,4,2):
            return 2
        case (9,5,2):
            return -2
        case (10,4,4):
            return 1
        case (10,5,5):
            return 1
        case (10,4,5):
            return -1
        case (10,5,4):
            return -1
        case (11,4,0):
            return 1
        case (11,5,1):
            return 1
        case (11,4,1):
            return -1
        case (11,5,0):
            return -1
        case (12,0,0):
            return 1
        case (12,0,1):
            return 1
        case (12,1,0):
            return -1
        case (12,1,1):
            return -1
        case (13,0,0):
            return -(1 + 1j)
        case (13,0,1):
            return -(1 + 1j)
        case (13,1,0):
            return (1 + 1j)
        case (13,1,1):
            return (1 + 1j)
        case (13,0,2):
            return 2
        case (13,1,2):
            return -2
        case (13,0,4):
            return 1j
        case (13,0,5):
            return 1j
        case (13,1,4):
            return -1j
        case (13,1,5):
            return -1j
        case (14,0,4):
            return 1
        case (14,1,5):
            return 1
        case (14,0,5):
            return -1
        case (14,1,4):
            return -1
        case (15,0,0):
            return 1
        case (15,1,1):
            return 1
        case (15,0,1):
            return -1
        case (15,1,0):
            return -1
        case _:
            return 0

For any two-qubit input state $\rho$, decomposes $\rho$ into the Pauli operator basis $E_i$

In [ ]:
def decompose_state_into_pauli_basis_process2(rho, should_print_description = False):

    if not ishermitian(rho):
        print(f"Matrix rho is not Hermitian")
    else:
        if not is_pos_semidef(rho):
            print(f"Matrix rho is not positive semi-definite")
    if not np.isclose(np.trace(rho), 1.0):
        print("Matrix rho is not normalised")

    coefficients = []

    if should_print_description:
        print("Input state rho decomposed in the Pauli basis E(i), i.e. rho = \sum a(i)E(i) is:\n")
    
    for j in range(len(rho[0])**2):
        result = 0.25 * np.trace((func_E2(j).conj().T) @ rho)

        if not np.isclose(result.imag, 0.0):
            raise ValueError(f"Result {result} should have no imaginary part")
        else:
            result = result.real

        coefficients.append(result)

        if should_print_description:
            print(f"{j}) a({j}) = 1/4 * Tr[E({j}).rho] = {result}")
    
    return coefficients

Test for $\rho = \left | 0 \right >\left | 0 \right > \left < 0 \right |\left < 0 \right |$

In [ ]:
rho_test = np.array([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

decompose_state_into_pauli_basis_process2(rho_test, True)

The following describes decomposing an input Pauli operator $E_j$ into a linear combination of two qubit Bloch sphere states $\mathcal{B}_i \otimes \mathcal{B}_j \left | 0 \right > \left | 0 \right > \left < 0 \right | \left < 0 \right | \mathcal{B}^{\dagger}_i \otimes \mathcal{B}^{\dagger}_j$

In [ ]:
def describe_decomposing_pauli_operator_into_rank_1_bloch_state2(k, should_print_description):

    if should_print_description:
        print(f"Input Pauli operator E({k}) decomposed into the rank 1 Bloch sphere states, i.e. E({k}) = \sum c({k},i,j)F(i)⊗F(j)|0><0|F(i)^dagger ⊗ F(j)^dagger, is:\n")

    for i in range(6):
        for j in range(6):

            if should_print_description:
                print(f"{i},{j}) Coefficient of F({i})F({j}) term is: c({k},{i},{j}) = {func_c2(k,i,j)}\n")

In [ ]:
describe_decomposing_pauli_operator_into_rank_1_bloch_state2(1, True)

Helper functions to check math

In [ ]:
def reconstruct_pauli_from_rank_1_bloch_states2(i):

    answer = 0.0 + 0.0*1j

    for j in range(6):
        for k in range(6):
            answer += func_c2(i, j, k) * (kron(func_B(j), func_B(k)) @ np.array([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]) @ (kron(func_B(j).conj().T, func_B(k).conj().T)))
    
    return answer

def check_EC_mapping2(i):

    reconstructed_operator = reconstruct_pauli_from_rank_1_bloch_states2(i)

    if not np.allclose(reconstructed_operator, func_E2(i)):

        result = False

        print(f"Reconstruction for operator E({i}) failed!\n")
    
    else:
        result = True

        print(f"Reconstruction for operator E({i}) passed!\n")
    
    return result

for j in range(16):

    check_EC_mapping2(j)

Corresponds to $B_{ijklmn}$ in Eq 7 above

In [ ]:
def B_Bloch2(i, j, k, l, m, n):

    # start from qubits ground states, create one of six Bloch sphere states
    # using func_B for each of the two qubits, add one of the 16 (4**2)
    # Pauli operator basis func_E, and measure one of the six Bloch sphere
    # state projectors for each qubit, i.e.
    # func_B[.]^{\dagger}func_B[.]^{\dagger} |0>|0><0|<0| func_B[.] func_B[.].
    # These B_Bloch constants appear when expressing the Pauli states
    # in terms of combinations of Bloch sphere states, assuming you
    # start from the qubits' ground states

    starting_state = np.array([[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

    bloch_state = np.kron(func_B(i), func_B(j)) @ starting_state @ np.kron(func_B(i).conj().T, func_B(j).conj().T)
    add_paulis = func_E2(m) @ bloch_state @ (func_E2(n).conj().T)
    measure_bloch_state = np.kron(func_B(k).conj().T, func_B(l).conj().T) @ starting_state @ np.kron(func_B(k), func_B(l)) @ add_paulis

    return measure_bloch_state.trace()

Corresponds to $P_{stmn}$ in Eq. 2 above, and explicitly performs the mapping of Eq. 9

In [ ]:
def P_Pauli2(s, t, m, n):

    result = 0

    for i in range(0, 6):
        for j in range(0, 6):
            for k in range(0, 6):
                for l in range(0, 6):
                    result += func_c2(s, i, j) * np.conj(func_c2(t, k, l)) * B_Bloch2(i, j, k, l, m, n)

    return result

Print out a description of Eq 9 per index

In [ ]:
def convert_Matrix_from_bloch_to_pauli2(should_print_description):

    if should_print_description:

        print(f"The P matrix, converted from the B matrix (i.e. from Bloch states to Paulis), is:\n")
    
        for s in range(16):
            for t in range(16):
                for m in range(16):
                    for n in range(16):
                        print(f"The matrix element Tr[E^dagger({t})E({m})E({s})E^dagger({n})] is: P({s},{t},{m},{n}) = {P_Pauli2(s, t, m, n)}")

#### Note: computing the PMatrix in the two-qubit case takes a while. But since the matrix is a constant, I've computed it once and it can simply be loaded from the accompanying pickle file

In [ ]:
# commented out since this just prints out a description of the many measurements, and takes a while
# convert_Matrix_from_bloch_to_pauli2(True)

Assuming you didn't load the file, the last line reshapes $P_{stmn}$ into a 2D matrix (used to solve Eq. 2)

In [ ]:
file_string = "/Users/nbornman-local/Desktop/PMatrix2.pickle"

if os.path.isfile(file_string):
    with open(file_string, "rb") as file:
        PMatrix2 = pickle.load(file)
else:
    PMatrix2 = np.array([[P_Pauli2(np.floor(v/16).astype(int), v % 16, np.floor(w/16).astype(int), w % 16) for w in range(16**2)] for v in range(16**2)]).round(8)

### Two qubit example: amplitude damping on qubit 1 and de-phasing on qubit 2

Simulate amplitude damping noise channel on qubit 1 and dephasing noise on qubit 2, with probability to damp, $p0$, equal to $13\%$, and the probability of dephasing, $p1$, equal to $20\%$ (see Ref 3)

In [ ]:
p0 = 0.13
p1 = 0.2

k0 = kron(np.array([[1, 0], [0, np.sqrt(1-p0)]]), np.sqrt(1- p1/2) * i)
k1 = kron(np.array([[0, np.sqrt(p0)], [0, 0]]), np.sqrt(1- p1/2) * i)
k2 = kron(np.array([[1, 0], [0, np.sqrt(1-p0)]]), np.sqrt(p1/2) * z)
k3 = kron(np.array([[0, np.sqrt(p0)], [0, 0]]), np.sqrt(p1/2) * z)

The process in terms of Kraus operators

In [ ]:
def damping_and_dephasing(rho):

    return (k0 @ rho @ (k0.conj().T)) + (k1 @ rho @ (k1.conj().T)) + (k2 @ rho @ (k2.conj().T)) + (k3 @ rho @ (k3.conj().T))

Initialise empty Bloch sphere basis measurement array 'lamb' (correspondint to $\lambda$), and Pauli basis array 'm' (corresponding to $m$).

In [ ]:
lamb = np.empty((6, 6, 6, 6))
m = np.empty((16, 16))

Simulate the process of performing two qubit process tomography for the input 'process', by preparing and measuring Bloch sphere states

In [ ]:
def simulate_process_tomography_measurements_with_input_bloch_states_and_measuring_bloch_projectors2(process, should_print_description):

    for i in range(6):
        for j in range(6):
            for k in range(6):
                for l in range(6):

                    rho = np.array([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])

                    rho = kron(func_B(i), func_B(j)) @ rho @ kron((func_B(i).conj().T), (func_B(j).conj().T))

                    rho = process(rho)

                    rho = kron((func_B(k).conj().T), (func_B(l).conj().T)) @ np.array([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]) @ kron(func_B(k), func_B(l)) @ rho

                    res = np.trace(rho)

                    if not np.isclose(res.imag, 0.0):
                        raise ValueError(f"Result {res} should be real")
                    else:
                        res = res.real

                    lamb[i, j, k, l] = res

                    if should_print_description:

                        print(f"Measurement result of input process for the input state F({i})⊗F({j})|0> and Bloch projector F({k})⊗F({l})|0><0|Fdag({k})⊗Fdag({l}) = {lamb[i, j, k, l]}\n")

In [ ]:
simulate_process_tomography_measurements_with_input_bloch_states_and_measuring_bloch_projectors2(damping_and_dephasing, True)

For a given input Bloch sphere measuremenet matrix (here called 'lamb'), describes the corresponding Pauli measurement matrix (here 'm')

In [ ]:
def map_from_bloch_state_measurements_to_pauli2(arr, should_print_description):

    if arr.shape != (6, 6, 6, 6):
        raise ValuerError("Input array must be 6x6x6x6")
    
    if should_print_description:
        print(f"For the Bloch states' measurements, the Pauli basis m measurement matrix's elements are:\n")

    for q in range(16):
        for n in range(16):

            result = 0
    
            for i in range(0, 6):
                for j in range(0, 6):
                    for k in range(0, 6):
                        for l in range(0, 6):

                            result += func_c2(q, i, j) * np.conj(func_c2(n, k, l)) * arr[i][j][k][l]
            

            if not np.isclose(result.imag, 0.0):
                raise ValueError(f"Result {result} should be real")
            else:
                result = result.real
            
            m[q, n] = result
            
            if should_print_description:
                print(f"If the Pauli state E({q}) were prepared and the Pauli observable E({n}) measured, the measured value would be: Tr(E^(dagger)({n})ε(E({q}))) = m({q},{n}) = {result}")

In [ ]:
map_from_bloch_state_measurements_to_pauli2(lamb, True)

Reshapes Pauli measurement matrix 'm' into a 1D vector.

In [ ]:
measurement_vector = np.array([m[np.floor(v/16).astype(int), v % 16] for v in range(16**2)])

Solves Eq. 2 for $\chi$ in vectorised form, and reshapes the vector into the $\chi_{mn}$ matrix

In [ ]:
chi_vector = solve(PMatrix2, measurement_vector)
chi_matrix = chi_vector.reshape(16, 16)

Plot the $\chi$ matrix

In [ ]:
def plot_process_tomography2(chi_vector, heading = r'$\chi$ matrix', save_file: str = None):

    plt.rcParams['text.usetex'] = True

    ticksfont = {'color':  'black', 'weight': 'normal','size': 4}

    cmap = colormaps.get_cmap('viridis')

    # set up figure
    fig = plt.figure(figsize=(9, 4), dpi=250, facecolor="white")
    ax = fig.add_subplot(111, projection='3d')

    # coordinates
    r = np.arange(16)
    _x, _y = np.meshgrid(r, r)
    x, y = _x.ravel(), _y.ravel()

    values0 = np.abs(chi_vector)
    values1 = np.angle(chi_vector)

    top = values0
    bottom = np.zeros_like(top)

    width = depth = 0.7

    norm = Normalize(vmin=-np.pi, vmax=np.pi)
    colors = cmap(norm(values1))

    xy_ticks_labels = [r"$II$", r"$IX$", r"$IY$", r"$IZ$", r"$XI$", r"$XX$", r"$XY$", r"$XZ$", r"$YI$", r"$YX$", r"$YY$", r"$YZ$", r"$ZI$", r"$ZX$", r"$ZY$", r"$ZZ$"]

    subplot = ax.bar3d(x, y, bottom, width, depth, top, shade=True, color=colors)
    ax.set_xticks(r + 0.5, labels=xy_ticks_labels, fontdict=ticksfont)
    ax.set_yticks(r + 0.5, labels=xy_ticks_labels, fontdict=ticksfont)
    ax.set_zticks([0, (max(top)/2).round(2), max(top).round(2)])
    ax.set_xlabel("Prepared", fontdict = axisfont)
    ax.set_ylabel("Measured", fontdict = axisfont)
    ax.set_title(heading, fontdict = titlefont)
    ax.view_init(20, -60, 0)

    sc = cm.ScalarMappable(cmap=cmap,norm=norm)
    cbar = plt.colorbar(sc, ax=ax, pad=0.1, shrink=0.7)
    cbar.set_ticks(ticks=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi], labels=[r"$-\pi$", r"$-\pi/2$", r"0", r"$\pi/2$", r"$\pi$"])

    if save_file:
        plt.savefig(save_file, bbox_inches='tight')

    plt.show()

In [ ]:
plot_process_tomography2(chi_vector)

Check trace-preserving-ness: if $\chi$ is trace-preserving, it must satisfy $\sum_{m,n=0}^{d^2 - 1} \chi_{mn} E_{n} E_m = I$

In [ ]:
sum_of_terms = np.sum([chi_matrix[m, n] * func_E2(n) @ func_E2(m) for n in range(16) for m in range(16)], axis=0)

In [ ]:
if np.allclose(sum_of_terms, np.eye(4)):
    print("Process is trace preserving")
else:
    print("Process is not trace preserving")

Given $\chi$ matrix, compute the corresponding Kraus operators

In [ ]:
def compute_kraus_operators(chi_matrix, should_print_description):

    print("NOTE: NEED TO DEBUG THIS - CHECK WHETHER chi_matrix NEEDS TO BE UNITARY OR WHETHER THIS SCHEME IS VALID REGARDLESS")

    kraus_operators = []

    eigenvalues, eigenvectors = eig(chi_matrix)

    if not np.allclose(eigenvectors @ (eigenvectors.conj().T), np.eye(16)):
        print("Similarity matrix is not unitary!")
    
    for m in range(16):
        
        operator = np.sqrt(eigenvalues[m]) * np.sum([eigenvectors[j, m]*func_E2(j) for j in range(16)], axis=0)

        if should_print_description and not np.allclose(operator, np.zeros((4,4))):
            print(f"Kraus operator for arbitrary index m = {m}: {operator}")
            kraus_operators.append(operator.round(6))
    
    return kraus_operators

In [ ]:
compute_kraus_operators(chi_matrix, True)